# Spatial Visualization with ArcGIS API for Python

## arcgis.mapping module

The arcgis.mapping module contains classes and functions to represent and interact with web maps, scenes, and certain layer types such as map image and vector tiles. In this page we will observe how to visualize maps, scenes and layers using the map widget in Jupyter notebook environment.

- Setting the map properties
    + Zoom level
    + Map center
    + Basemaps
    + 3D Mode
- Adding layers to the map
- Adding Item objects to the map
- Adding layer objects to the map
- Adding Imagery layers
- Listing and removing layers from the map
- Graphic support
- Saving the map as a web map

## Creating a Map

In [ ]:
from arcgis.gis import GIS
gis = GIS()
m = gis.map('Frankfurt, DE')
m

## WebMap Properties

  + Zoom level
  + Map center
  + Basemaps
  + 3D Mode
  + rotation

### Zoom Level

In [ ]:
m.zoom

In [ ]:
m.zoom = 15

### Rotation

- values are 0-360 degrees

In [ ]:
import time
for r in [45, 90, 180, 270, 0]:
    m.rotation = r
    time.sleep(3)

### Basemaps

- provide the underlying background for the web map
- `basemaps` provides a list available basemap 
- `basemap` - get/set the background

In [ ]:
m1 = gis.map('Paris') # RUN ME :) 
m1

In [ ]:
for bm in m1.basemaps[::2]:
    print(m1.basemap)
    m1.basemap = bm
    time.sleep(1.5)

### 3D Mode

- The map widget supports both 2D and 3D

In [ ]:
m2 = gis.map('Zurich', mode='3D')
m2.zoom = 8
m2

## Adding Content to the Map Widget

In [ ]:
flayer_search_result = gis.content.search("europe owner:esri_dm","Feature Layer")
flayer_search_result

#### Adding by Item

- Data can be displayed on the webmap using Item
- This supports: Feature Layer, Imagery Services and other.

In [ ]:
m3 = gis.map('Netherlands')
m3.zoom = 7
m3

In [ ]:
m3.add_layer(flayer_search_result[0])

In [ ]:
img_items = gis.content.get("e7e530e0067d47c7b3478d44518b563e")
img_items

In [ ]:
m3.add_layer(img_items)

In [ ]:
m3.remove_layers()

#### Adding by Layer

In [ ]:
m4 = gis.map("Netherlandas")
m4.zoom = 7
m4

In [ ]:
img_items.layers

In [ ]:
m4.add_layer(img_items.layers[0])

#### Adding a Spatially enabled DataFrame

In [ ]:
item = gis.content.get("6996f03a1b364dbab4008d99380370ed")
sdf = item.layers[0].query(as_df=True)

In [ ]:
sdf.head()

In [ ]:
sdf.spatial.plot(map_widget=m4)

## Cartography using the ArcGIS API from Python

### Smart Mapping

- process of creating maps with ease
- uses algorithms to auto generate the best cartography

In [ ]:
freeway_item = gis.content.get('91c6a5f6410b4991ab0db1d7c26daacb')
freeway_item

In [ ]:
m5 = gis.map("Los Angles, CA")
m5

In [ ]:
m5.center = [34, -118]
#m5.zoom = 11
m5.add_layer(freeway_item, {"renderer":"ClassedSizeRenderer", "field_name": "DIST_MILES"})

#### Visualize by Field


In [ ]:
m6 = gis.map('Washington State')
m6

In [ ]:
m6.add_layer({"type":"FeatureLayer", 
               "url":"http://sampleserver6.arcgisonline.com/arcgis/rest/services/Census/MapServer/2",
               "definition_expression" : "STATE_NAME='Washington'",
               "renderer":"ClassedColorRenderer",
               "field_name":"POP2007",
               "opacity":0.7})

In [ ]:
m6.legend = True

### Advanced Cartography

#### Compare A to B

- Renderers can compare two values

In [ ]:
from arcgis.features import FeatureLayer

map4 = gis.map("France", 2)

map4_renderer = {"renderer": "autocast", #This tells python to use JS autocasting
                 "type": "classBreaks",  
                 "field":"POP2025",
                 "transparency":80,
                 "minValue":-9007199254740991}

map4_renderer["visualVariables"] = [{   "type": "colorInfo",
                                        "field": "POP2025",
                                        "normalizationField": "POP2015",
                                        "stops": [
                                            {   "value": 1.12,
                                                "color": [230,97,1,255],
                                                "label": "< 112%"
                                            },
                                            {   "value": 1.1800000000000002,
                                                "color": [253,184,99,255],
                                                "label": "112% ~ 124%"
                                            },
                                            {   "value": 1.24,
                                                "color": [247,247,247,255],
                                                "label": "124%"
                                            },
                                            {   "value": 1.3,
                                                "color": [178,171,210,255],
                                                "label": "124% ~ 136%"
                                            },
                                            {   "value": 1.36,
                                                "color": [94,60,153,255],
                                                "label": "> 136%"
                                            }]
                                    }]

map4_renderer["classBreakInfos"] = [{   "symbol": {
                                            "color": [170,170,170,255],
                                            "size": 10.5,
                                            "angle": 0,
                                            "xoffset": 0,
                                            "yoffset": 0,
                                            "type": "esriSMS",
                                            "style": "esriSMSCircle",
                                            "outline": {
                                                "color": [153,153,153,255],
                                                "width": 0.375,
                                                "type": "esriSLS",
                                                "style": "esriSLSSolid"
                                            }
                                        },
                                        "classMaxValue": 9007199254740991
                                    }]

world_flayer = FeatureLayer("http://services.arcgis.com/nGt4QxSblgDfeJn9/arcgis/rest/services/World_LargestUrbanAreas_fs/FeatureServer/0")

map4.add_layer(world_flayer,
               { "type": "FeatureLayer",
                 "renderer": map4_renderer,
                 "field_name":"POP2025",
                 "normalizationType": "esriNormalizeByField",
                 "normalizationField": "POP2015"})

map4

In [ ]:
map4.legend = True

#### Render on Numeric Attribute


In [ ]:
from arcgis.features import FeatureLayer
usa_flayer = FeatureLayer("https://services.arcgis.com/V6ZHFr6zdgNZuVG0/arcgis/rest/services/USA_County_Crops_2007/FeatureServer/0")
map1 = gis.map()
map1.center = [38.32016202887768, -94.49274418999963]
map1.zoom = 4
map1.add_layer(usa_flayer, { "type": "FeatureLayer",
                             "renderer":"ClassedColorRenderer",
                             "field_name":"M086_07"})
map1

### Rendering Spatially enabled DataFrame

The Spatially Enabled Dataframe has a plot() method that uses a syntax and symbology similar to matplotlib for visualizing features on a map. With this functionality, you can easily visualize aspects of your data both on a map and on a matplotlib chart using the same symbology!

Some unique characteristics of working with the visualization capabalities on the SDF:

    - Uses Pythonic syntax
    - Uses the same syntax as visualizing charts on Pandas DataFrames
    - Uses symbology familiar to users of matplotlib
    - Works on features and attributes simultaneously, eliminating to a great extent the need to iterate over all features (rows)
    - Handles reading and writing to multiple formats aiding data conversion
    

<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;}
.tg td{font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;border-color:black;}
.tg th{font-family:Arial, sans-serif;font-size:14px;font-weight:normal;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;border-color:black;}
.tg .tg-1wig{font-weight:bold;text-align:left;vertical-align:top}
.tg .tg-fymr{font-weight:bold;border-color:inherit;text-align:left;vertical-align:top}
.tg .tg-0pky{border-color:inherit;text-align:left;vertical-align:top}
.tg .tg-0lax{text-align:left;vertical-align:top}
</style>
<table class="tg">
  <tr>
    <th class="tg-fymr">Renderer</th>
    <th class="tg-1wig">Syntax</th>
    <th class="tg-1wig">Explanition</th>
  </tr>
  <tr>
    <td class="tg-0pky">Simple</td>
    <td class="tg-0lax">'s'</td>
    <td class="tg-0lax">renders using one symbol only</td>
  </tr>
  <tr>
    <td class="tg-0pky">Unique</td>
    <td class="tg-0lax">'u'</td>
    <td class="tg-0lax">renders each unique value with different symbol</td>
  </tr>
  <tr>
    <td class="tg-0lax">Unique</td>
    <td class="tg-0lax">'u-a'</td>
    <td class="tg-0lax">renders each unique value with a different symbol using an arcade expression</td>
  </tr>
  <tr>
    <td class="tg-0lax">Class Breaks</td>
    <td class="tg-0lax">'c'</td>
    <td class="tg-0lax">renders each group of values with a different color or size</td>
  </tr>
  <tr>
    <td class="tg-0lax">Heatmap</td>
    <td class="tg-0lax">'h'</td>
    <td class="tg-0lax">renders density of point data as a raster </td>
  </tr>
</table>

#### Obtain Test Data

In [ ]:
from arcgis.gis import GIS
item = gis.content.get("85d0ca4ea1ca4b9abf0c51b9bd34de2e")
flayer = item.layers[0]
sdf = flayer.query(where="AGE_45_54 < 1500", as_df=True)
sdf.head()

#### Unique Renderer

In [ ]:
gis = GIS()
map4a = gis.map('Reno, NV', zoomlevel=4)
map4a.center = [38.365564106215935, -94.14943750000482]
map4a

In [ ]:
sdf.spatial.plot(map_widget=map4a,
                renderer_type='u',
                col='ST')

#### Simple Renderer

In [ ]:
sdf.spatial.plot(symbol_type='simple',
                 symbol_style='d', # d - for diamonds
                 colors='Reds_r',
                 cstep=10,
                 outline_color='Blues',
                 marker_size=10)

#### Classbreak Renderer

In [ ]:
sdf.spatial.plot(renderer_type='c',  # for class breaks renderer
                 class_count=20,  
                 col='POPULATION',
                 cmap='prism',
                 alpha=0.7)

#### Understanding Colormaps

- The display_colormaps function provides a quick, easy way to visualize the pre-defined set of colormaps you can use.

In [ ]:
from arcgis.mapping import display_colormaps
display_colormaps()

In [ ]:
from arcgis.mapping import symbol

colormaps = symbol.ALLOWED_CMAPS
for a,b,c,d,e in zip(colormaps[::5], colormaps[1::5], colormaps[2::5], colormaps[3::5], colormaps[4::5]):
    print("{:<20}{:<20}{:<20}{:<20}{:<}".format(a,b,c,d,e))

In [ ]:
from arcgis.mapping import symbol
symbol.create_symbol?